In [5]:
import os
import pandas as pd
import statistics
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from google.cloud import bigquery
from google.oauth2 import service_account
from google.cloud.bigquery import magics

In [6]:
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] ='service-account.json'
bigquery_client = bigquery.Client(project='ironhacks-covid19-data')
bigquery_client
bigquery_client = bigquery.Client()

QUERY = """

SELECT *
FROM ironhacks-covid19-data.ironhacks_covid19_competition.predictionset_covid19_phase4

"""

query_job = bigquery_client.query(QUERY)
data = query_job.to_dataframe()
data.head()


,uuid,brand_name,naics,top_category,week_number,date_start,nr_POIs,cards_per_week,transactions_per_week,total_spent_per_week,avg_distance_from_home,sd_distance_from_home,avg_median_dwell,sd_median_dwell,sum_visit_counts
0,c06b28c5-5506-4399-b6fc-4817c3d1f4f6,U-Haul,532120,Automotive Equipment Rental and Leasing,10,2020-03-09,16,329,338,29737.29,6941,2041,28,44,421
1,3800c466-9f63-4e63-ab70-29b30869d583,Dave & Buster's,722410,Drinking Places (Alcoholic Beverages),10,2020-03-09,1,187,280,12783.41,23187,0,179,0,47
2,7ed193b5-28a1-4407-85dd-d53f25907580,Pandora,448310,"Jewelry, Luggage, and Leather Goods Stores",10,2020-03-09,1,28,28,232.82,6728,0,13,0,7
3,31ebfa4c-ad2b-4915-b9ff-74d7ea321e60,GNC (General Nutrition Centers),446191,Health and Personal Care Stores,10,2020-03-09,38,290,295,13909.27,9999,12608,23,24,1441
4,4dafbc3c-f718-4765-b8f2-328222e51e0d,Orscheln Farm & Home,424910,Miscellaneous Nondurable Goods Merchant Wholes...,10,2020-03-09,10,92,96,3792.59,10977,6377,22,15,384


In [9]:
'''
Solution:
Create a moving average for each uuid with a window of 3 to accurately predict from week to week using solely sum_visit_counts
in a time series model


My solution utilizes a moving average with a time series model to best predict the number of visits that a brand has. 
Since I only need the dates and outcome for a time series model, I can eliminate the other variables. So as more data comes in,
I can adjust the window of the moving average to accommodate and improve on the model more naturally instead of having a complete
remodel of my solution. This way, I can work on improving the solution instead of having to rework the model. 
'''


column_names = ["uuid", "week_number", "sum_visit_counts"]

df = pd.DataFrame(columns = column_names)



for brand_name, data_brand in data.groupby('uuid'):
    if len(data_brand.index) > 2:
        frame = data_brand
        uuid = data_brand['uuid'].values[0]
        frame.sort_values(by=['week_number'], inplace = True)
        #print(uuid)
        frame.drop(['uuid'], axis=1, inplace = True)
        frame.drop(['week_number'], axis=1, inplace = True)
        frame.drop(['brand_name'], axis=1, inplace = True)
        frame.drop(['top_category'], axis=1, inplace = True)
        frame.drop(['nr_POIs'], axis=1, inplace = True)
        frame.drop(['naics'], axis=1, inplace = True)
        frame.drop(['cards_per_week'], axis=1, inplace = True)
        frame.drop(['transactions_per_week'], axis=1, inplace = True)
        frame.drop(['total_spent_per_week'], axis=1, inplace = True)
        frame.drop(['avg_distance_from_home'], axis=1, inplace = True)
        frame.drop(['sd_distance_from_home'], axis=1, inplace = True)
        frame.drop(['avg_median_dwell'], axis=1, inplace = True)
        frame.drop(['sd_median_dwell'], axis=1, inplace = True)
        
        rolling = frame.rolling(window=2)
        rolling_mean = rolling.mean()
    #     plt.plot(data_brand['date_start'], data_brand['sum_visit_counts'])
    #     plt.plot(data_brand['date_start'], rolling_mean[-100:])

        X = frame.values
        window = 2
        X = np.delete(X, 0, axis=1)
        
        history = [X[i][0] for i in range(window)]
        test = [X[i] for i in range(window, len(X))]
        predictions = []

            # walk forward over time steps in test
        for t in range(len(test)):
            length = len(history)
            yhat = np.mean([history[i] for i in range(length-window,length)])
            obs = test[t]
            predictions.append(yhat)
            history.append(obs)
            #print('predicted=%f, expected=%f' % (yhat, obs))
        #plt.plot(test)
        #plt.plot(predictions, color='red')
        #plt.show()
        pred = int(predictions[0])
        temp = pd.DataFrame({"uuid":uuid, 
                            "week_number":21,  
                            "sum_visit_counts":pred}, index = [0])
        df = df.append(temp)
    else:
        temp = pd.DataFrame({"uuid":uuid, 
                    "week_number":18,  
                    "sum_visit_counts": float("nan")}, index = [0])
        df = df.append(temp)
    

df.head()

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/conda/lib/python3.7/site-packages/pandas/core/frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,uuid,week_number,sum_visit_counts
0,00681c7d-46b7-44e9-9b35-98331e53a204,21,34985
0,013b319a-9146-4623-a13d-a94db23ee41a,21,2273
0,015ae118-28e7-406d-b981-0bc32eb4859c,21,583
0,03d1a415-7627-4fa0-a02c-129a079b6e8d,21,154
0,04d0627e-0cb3-49eb-93dc-4707a45a3bdb,21,2255


In [11]:
df.to_csv(r'submission_prediction_output.csv', index = False)
